<a href="https://colab.research.google.com/github/BMateusSs/Projeto-PISI3/blob/main/exploratory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install dependencies as needed:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "winemag-data-130k-v2.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "zynicide/wine-reviews",
  file_path
)

<ipython-input-6-7195d7428f2b>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


ValueError: Error reading file: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

Limpeza dos dados

In [25]:
import pandas as pd

df = df.dropna(subset=["region_1"])
df = df[["region_1", "description"]]
df.head()
# 1. Identificar as 10 regiões com mais registros
top_10_regions = df['region_1'].value_counts().nlargest(10).index.tolist()

# 2. Filtrar o DataFrame para incluir apenas as top 10 regiões
df_top_10 = df[df['region_1'].isin(top_10_regions)]

# 3. Identificar a contagem mínima entre as top 10 regiões
min_count = df_top_10['region_1'].value_counts().min()

print(f"As 10 regiões com mais registros são: {top_10_regions}")
print(f"A contagem mínima de registros entre as top 10 regiões é: {min_count}")

# 4. Amostrar cada uma das top 10 regiões para ter a contagem mínima
sampled_dfs = []
for region in top_10_regions:
    df_region = df_top_10[df_top_10['region_1'] == region]
    # Amostra aleatoriamente 'min_count' registros para cada região
    df_sampled = df_region.sample(n=min_count, random_state=42) # random_state para reprodutibilidade
    sampled_dfs.append(df_sampled)

# Concatenar os DataFrames amostrados de volta em um único DataFrame
df_balanced = pd.concat(sampled_dfs)

# Opcional: Embaralhar as linhas do DataFrame resultante
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar as primeiras linhas do DataFrame balanceado
print("\nPrimeiras linhas do DataFrame balanceado:")

# Verificar a contagem de registros por região no DataFrame balanceado
print("\nContagem de registros por região no DataFrame balanceado:")
print(df_balanced['region_1'].value_counts())
df_balanced.head()
df = df_balanced

As 10 regiões com mais registros são: ['Napa Valley', 'Columbia Valley (WA)', 'Russian River Valley', 'California', 'Paso Robles', 'Mendoza', 'Willamette Valley', 'Alsace', 'Champagne', 'Barolo']
A contagem mínima de registros entre as top 10 regiões é: 1599

Primeiras linhas do DataFrame balanceado:

Contagem de registros por região no DataFrame balanceado:
region_1
Barolo                  1599
Napa Valley             1599
Columbia Valley (WA)    1599
Paso Robles             1599
California              1599
Mendoza                 1599
Champagne               1599
Alsace                  1599
Willamette Valley       1599
Russian River Valley    1599
Name: count, dtype: int64


In [26]:
import nltk
from nltk.corpus import stopwords
import string

# Baixe as stopwords se ainda não as tiver
try:
    stopwords.words('portuguese')
except LookupError:
    nltk.download('stopwords')

stop_words = set(stopwords.words('portuguese'))

def clean_text(text):
    # Converter para lowercase
    text = text.lower()
    # Remover pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remover stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Aplicar a função de limpeza na coluna 'description'
df['description'] = df['description'].apply(clean_text)
df.head()

,region_1,description
0,Barolo,enticing aromas of red rose wild red berry and...
1,Barolo,underbrush grilled porcini red berry and grill...
2,Napa Valley,lovely cabernet richly attractive and deeply f...
3,Columbia Valley (WA),this new winery knocks it out of the park with...
4,Paso Robles,this untraditional blend works well with the l...


In [ ]:
from sentence_transformers import SentenceTransformer
# Experimente com um modelo diferente, por exemplo:
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
embeddings = model.encode(df['description'].tolist(), show_progress_bar=True)


Batches:   0%|          | 0/500 [00:00<?, ?it/s]